In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split
import config
import random
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.neural_network import MLPClassifier
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
from matplotlib import pyplot
from keras.callbacks import History 
import h5py

In [23]:
def read_files():
    tfidf_df = pd.read_csv(config.datasets_dir + config.optimized_dataset)
    #print(list(tfidf_df.columns)[:30])
    #clean_df = pd.read_csv(config.datasets_dir + config.optimized_dataset)
    df = tfidf_df
    #df['ASSET_CLASS'] = clean_df['ASSET_CLASS']
    #df['ASSET_CLASS'] = pd.Categorical(df['ASSET_CLASS'])
    #df['ASSET_CLASS_CODES'] = df['ASSET_CLASS'].cat.codes
    return df

In [24]:
def scores(y_pred, Y_test):
    print('Accuracy:   '+str(accuracy_score(y_pred, Y_test)))
    print('Precision Macro:   '+ str(precision_score(y_pred, Y_test,average = 'macro')))
    print('Recall Macro:     '+str(recall_score(y_pred, Y_test, average = 'macro')))
    print('F1 Score Macro:     '+str(f1_score(y_pred, Y_test, average = 'macro')))
    print('\n')

In [25]:
def trainTestSplit(df,n):
    random.seed(123)
    df1 = df['ASSET_CLASS'].value_counts().rename_axis('Assets').reset_index(name = 'counts')
    df_new = df1[df1['counts']>=n] # Train Test split 75% - train
    assets = list(df_new['Assets'])
    dffiltered = df[df['ASSET_CLASS'].isin(assets)]
    dffiltered['ASSET_CLASS_CODES'] = pd.Categorical(dffiltered['ASSET_CLASS'])
    dffiltered['ASSET_CLASS_CODES'] = dffiltered['ASSET_CLASS_CODES'].cat.codes
    x = dffiltered.drop(columns = ['ASSET_CLASS','ASSET_CLASS_CODES','important_words','BUSINESS_UNIT','PSC_CODE','FUND_SUBOBJCLASS','ORDER_DATE','ORDER_TITLE',                                                                                                                                                      
            'LINE_DESCRIPTION', 'VENDOR_NAME', 'VENDOR_COUNTRY', 'ASSET_CLASS_DESCRIPTION','text_fields','ASSET_CLASS_OLD','SUB_OBJ_DESCR','OBJ_CODE'])
    xcols = list(x.columns)
    y = dffiltered['ASSET_CLASS_CODES']
    X_train, X_test, Y_train, Y_test = train_test_split(x,y, test_size = 0.20, stratify = y)
    print(' Number of Assets ' + str(len(set(list(dffiltered['ASSET_CLASS'])))))
    #dict_codes = pd.Series(df.ASSET_CLASS.values, index = df.ASSET_CLASS_CODES).to_dict()
    return X_train, X_test, Y_train, Y_test

In [26]:

def MLP(X_train, X_test, Y_train, Y_test):
    print(X_train.shape)
    print(X_train.shape[1])

    model = Sequential()
    model.add(Dense(1000, input_dim = X_train.shape[1],activation = 'softmax'))
    model.add(Dense(750, activation = 'softmax'))
    model.add(Dense(500, activation = 'softmax'))
    model.add(Dense(61, activation = 'softmax'))
    # compile keras
    model.compile(loss='sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    model.summary()
    # Early stopping
    es = EarlyStopping(monitor = 'val_loss', verbose=3.1, patience = 20, min_delta = 1)
    # Fit the keras
    history = model.fit(X_train, Y_train, epochs = 500, batch_size = 32, validation_split = 0.1,verbose = 1,callbacks=[es])
    # Model Evaluation
    _, train_acc = model.evaluate(X_train, Y_train, verbose =2)
    _, test_acc = model.evaluate(X_test, Y_test, verbose=2)
    print('Train Accuracy  '+str(train_acc))
    print('Test Accuracy   '+str(test_acc))
    y_pred = model.predict_classes(X_test)
    scores(y_pred, Y_test)
    pyplot.plot(history.history['loss'], label = 'train')
    pyplot.plot(history.history['val_loss'], label = 'Validation')
    pyplot.xlabel('Epochs')
    pyplot.ylabel('Validation Loss')
    pyplot.legend()
    pyplot.show()
    
    # save best Model
    #mc = ModelCheckpoint(config.mlp_model_data2, monitor='val_loss',verbose=1, save_best_only=True)

    

In [31]:
def main():
    df = read_files()
    n = 100
    X_train, X_test, Y_train, Y_test = trainTestSplit(df,n)
    print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)
    #MLP(X_train, X_test, Y_train, Y_test)
    res = df['LINE_DESCRIPTION'].str.split().str.len().min()
    print(res)
    #print(len(Y_train.unique()))

In [32]:
main()

/snap/jupyter/6/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3296: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/snap/jupyter/6/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/snap/jupyter/6/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


 Number of Assets 61
(85524, 169) (21382, 169) (85524,) (21382,)
1.0


In [ ]:

print(max(sentences, key=len))
